## cyber~rank

In [57]:
import math
import collections
import functools
import operator

In [58]:
def rank_calc(cyberlinks: list, tolerance: float = 0.001, damping_factor: float = 0.8):
    cyberlinks_dict = dict(functools.reduce(operator.add, map(collections.Counter, cyberlinks)))
    objects = list(set([item for t in [list(x.keys())[0] for x in cyberlinks] for item in t]))
    rank = [0] * len(objects)
    size = len(objects)
    default_rank = (1.0 - damping_factor) / size
    dangling_nodes = [obj for obj in objects if obj not in [list(cyberlink.keys())[0][1] for cyberlink in cyberlinks]]
    dangling_nodes_size = len(dangling_nodes)
    inner_product_over_size = default_rank * (dangling_nodes_size / size)
    default_rank_with_correction = (damping_factor * inner_product_over_size) + default_rank
    change = tolerance + 1

    steps = 0
    prevrank = [0] * len(objects)
    while change > tolerance:
        for obj in objects:
            obj_index = objects.index(obj)
            ksum = 0
            income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == obj]
            for cyberlink in income_cyberlinks:
                linkStake = cyberlinks_dict[cyberlink]
                outcome_cyberlinks = [outcome_cyberlink for outcome_cyberlink in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlink[0] == cyberlink[0]]
                jCidOutStake = sum([cyberlinks_dict[outcome_cyberlink] for outcome_cyberlink in outcome_cyberlinks])
                if linkStake == 0 or jCidOutStake == 0:
                    continue
                weight = linkStake / jCidOutStake
                ksum = prevrank[obj_index] * weight + ksum
            rank[obj_index] = ksum * damping_factor + default_rank_with_correction
        change = abs(max(rank) - max(prevrank))
        prevrank = rank
        steps += 1
    res = list(zip(objects, rank))
    res.sort(reverse=True, key=lambda x: x[1])
    return res

In [59]:
# lets create 4 cyberlinks:
# the structure is {(particle_from, particle_to): weight}

cyberlinks = [
    {(1, 2): 1},
    {(1, 3): 2},
    {(4, 1): 3},
    {(4, 2): 4},
    {(2, 3): 3}
]

In [60]:
rank_calc(cyberlinks)

[(3, 0.13999999999999996),
 (2, 0.1034285714285714),
 (1, 0.08057142857142854),
 (4, 0.059999999999999984)]

## Resource tokens minting

In [12]:
def mint_resource_token(
                        hydrogen: int, 
                        investmint_time: int, 
                        base_vesting_resource: int = 10_000_000_000, 
                        base_vesting_time: int = 86400,
                        max_vesting_time: int = 604800):
    # hydrogen is a fuel token for resources minting
    # investment time is a time token locked
    if investmint_time > max_vesting_time:
        print('Impossible to lock tokens longer than max vesting time')
        return 0
    return int(hydrogen / base_vesting_resource) * (investmint_time / base_vesting_time)

In [13]:
mint_resource_token(100_000_000_000, 86400)

10.0

# Entropy calculation

In [14]:
def entropy_calc(cyberlinks: list, damping_factor: float = 0.8):
    cyberlinks_dict = dict(functools.reduce(operator.add, map(collections.Counter, cyberlinks)))
    objects = list(set([item for t in [list(x.keys())[0] for x in cyberlinks] for item in t]))
    swd = [0] * len(objects)
    sumswd = [0] * len(objects)
    entropy = [0] * len(objects)
    for i in range(len(swd)):
        income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == objects[i]]
        outcome_cyberlinks = [outcome_cyberlinks for outcome_cyberlinks in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlinks[0] == objects[i]]
        swd[i] = damping_factor * sum(cyberlinks_dict[income_cyberlink] for income_cyberlink in income_cyberlinks) + \
                 (1 - damping_factor) * sum(cyberlinks_dict[outcome_cyberlink] for outcome_cyberlink in outcome_cyberlinks)
    for i in range(len(sumswd)):
        income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == objects[i]]
        for income_cyberlink in income_cyberlinks:
            obj = objects.index(income_cyberlink[0])
            sumswd[i] += damping_factor * swd[obj]
        outcome_cyberlinks = [outcome_cyberlinks for outcome_cyberlinks in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlinks[0] == objects[i]]
        for outcome_cyberlink in outcome_cyberlinks:
            obj = objects.index(outcome_cyberlink[1])
            sumswd[i] += (1 - damping_factor) * swd[obj]
    for i in range(len(entropy)):
        if swd[i] == 0:
            continue
        income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == objects[i]]
        for income_cyberlink in income_cyberlinks:
            obj = objects.index(income_cyberlink[0])
            if sumswd[obj] == 0:
                continue
            entropy[i] += abs(-swd[i] / sumswd[obj] * math.log2(swd[i]/sumswd[obj]))
        outcome_cyberlinks = [outcome_cyberlinks for outcome_cyberlinks in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlinks[0] == objects[i]]
        for outcome_cyberlink in outcome_cyberlinks:
            obj = objects.index(outcome_cyberlink[1])
            if sumswd[obj] == 0:
                continue
            entropy[i] += abs(-swd[i] / sumswd[obj] * math.log2(swd[i] / sumswd[obj]))
    res = list(zip(objects, entropy))
    res.sort(reverse=True, key=lambda x: x[1])
    return res

In [15]:
cyberlinks = [
    {(1, 2): 8},
    {(1, 15): 6},
    {(3, 2): 16},
    {(4, 3): 8},
    {(5, 4): 4},
    {(5, 6): 6},
    {(7, 6): 6},
    {(8, 7): 4},
    {(9, 8): 4},
    {(9, 10): 4},
    {(10, 11): 4},
    {(11, 12): 6},
    {(12, 19): 9},
    {(13, 12): 6},
    {(13, 14): 8},
    {(15, 14): 12},
    {(15, 2): 12},
    {(16, 2): 8},
    {(16, 14): 8},
    {(17, 3): 8},
    {(17, 14): 8},
    {(18, 3): 8},
    {(18, 19): 6},
    {(20, 19): 6},
    {(21, 20): 4},
    {(21, 6): 6}
]

In [16]:
entropy_calc(cyberlinks)

[(6, 17.516289145081835),
 (2, 17.25863153266834),
 (14, 15.216844887355874),
 (3, 14.053600707437878),
 (19, 9.941154476005343),
 (8, 3.702235581737394),
 (10, 3.636335487718992),
 (12, 3.207173722192134),
 (11, 2.598279162340869),
 (7, 2.4909937454722426),
 (21, 1.048738318825225),
 (5, 1.0480304076460332),
 (15, 1.0452166294092318),
 (13, 0.9745569392632268),
 (17, 0.9429488462719593),
 (4, 0.9252119293765573),
 (18, 0.8972974707273671),
 (16, 0.8169213102506709),
 (20, 0.7851494605294047),
 (1, 0.768414547218145),
 (9, 0.64142134411275)]

In [17]:
rank_calc(cyberlinks)

[(2, 0.037988986070618715),
 (19, 0.036568837058632966),
 (6, 0.03429659863945577),
 (14, 0.033018464528668606),
 (3, 0.0330184645286686),
 (12, 0.026627793974732743),
 (7, 0.022367346938775505),
 (11, 0.022367346938775505),
 (8, 0.01739682539682539),
 (10, 0.01739682539682539),
 (15, 0.01668675089083252),
 (4, 0.01640272108843537),
 (20, 0.01640272108843537),
 (1, 0.01242630385487528),
 (5, 0.01242630385487528),
 (9, 0.01242630385487528),
 (13, 0.01242630385487528),
 (16, 0.01242630385487528),
 (17, 0.01242630385487528),
 (18, 0.01242630385487528),
 (21, 0.01242630385487528)]